In [3]:
# Install the CRAN version (0.7.1)
#install.packages("r5r")

# Install the latest version of the library (0.7.9)
devtools::install_github("ipeaGIT/r5r", subdir = "r-package")

sf (1.0-7 -> 1.0-9) [CRAN]


Installing 1 packages: sf

Warning message in i.p(...):
“installation of package ‘sf’ had non-zero exit status”
Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done



✔  checking for file ‘/tmp/RtmpgeIIFz/remotes204763333baaf/ipeaGIT-r5r-e463766/r-package/DESCRIPTION’
─  preparing ‘r5r’:
✔  checking DESCRIPTION meta-information
─  installing the package to process help pages
─  saving partial Rd database (1.8s)
─  checking for LF line-endings in source and make files and shell scripts
─  checking for empty or unneeded directories
─  building ‘r5r_0.7.90000.tar.gz’
   


In [4]:
# Allow 4 GB of memory
options(java.parameters = "-Xmx4G")

In [5]:
library(r5r)
library(sf)
library(data.table)
library(ggplot2)

Please make sure you have already allocated some memory to Java by running:
  options(java.parameters = '-Xmx2G').
You should replace '2G' by the amount of memory you'll require. Currently, Java memory is set to -Xmx4G



In [6]:
data_path <- system.file("extdata/poa", package = "r5r")

In [7]:
list.files(data_path)

[1] "fares"                      "poa_elevation.tif"         
[3] "poa_eptc.zip"               "poa_hexgrid.csv"           
[5] "poa_osm.pbf"                "poa_points_of_interest.csv"
[7] "poa_trensurb.zip"

In [8]:
poi <- fread(file.path(data_path, "poa_points_of_interest.csv"))

In [9]:
head(poi)

id,lat,lon
<chr>,<dbl>,<dbl>
public_market,-30.02756,-51.22781
bus_central_station,-30.02329,-51.21886
gasometer_museum,-30.03404,-51.24095
santa_casa_hospital,-30.03043,-51.22240
townhall,-30.02800,-51.22865
piratini_palace,-30.03363,-51.23068


In [10]:
points <- fread(file.path(data_path, "poa_hexgrid.csv"))

# sample points
sampled_rows <-  sample(1:nrow(points), 200, replace=TRUE)
points <- points[ sampled_rows, ]
head(points)

V1,id,lon,lat,population,schools,jobs,healthcare
<int>,<chr>,<dbl>,<dbl>,<int>,<int>,<int>,<int>
724,89a90129eb7ffff,-51.16364,-30.03674,494,0,181,0
639,89a901295bbffff,-51.15914,-30.07596,796,0,2,0
899,89a90129177ffff,-51.15196,-30.04883,2543,0,25,0
353,89a90129ddbffff,-51.18503,-30.03908,1430,0,194,0
618,89a90128097ffff,-51.20182,-30.07787,437,0,16,0
288,89a90128c53ffff,-51.22804,-30.04929,99,0,3153,0


In [11]:
# Indicate the path where OSM and GTFS data are stored
r5r_core <- setup_r5(data_path = data_path)



Finished building network.dat at /home/hentenka/.conda/envs/mamba/envs/r5edu/lib/R/library/r5r/extdata/poa/network.dat



In [12]:
# set departure datetime input
departure_datetime <- as.POSIXct("13-05-2019 14:00:00",
                                 format = "%d-%m-%Y %H:%M:%S")

# calculate accessibility
access <- accessibility(r5r_core = r5r_core,
                        origins = points,
                        destinations = points,
                        opportunities_colnames = c("schools", "healthcare"),
                        mode = c("WALK", "TRANSIT"),
                        departure_datetime = departure_datetime,
                        decay_function = "step",
                        cutoffs = 60
                        )

In [13]:
head(access)

id,opportunity,percentile,cutoff,accessibility
<chr>,<chr>,<int>,<int>,<dbl>
89a90129eb7ffff,schools,50,60,27
89a90129eb7ffff,healthcare,50,60,16
89a901295bbffff,schools,50,60,36
89a901295bbffff,healthcare,50,60,17
89a90129177ffff,schools,50,60,21
89a90129177ffff,healthcare,50,60,12


In [14]:
# set inputs
mode <- c("WALK", "TRANSIT")
max_walk_time <- 30 # minutes
max_trip_duration <- 120 # minutes
departure_datetime <- as.POSIXct("13-05-2019 14:00:00",
                                 format = "%d-%m-%Y %H:%M:%S")

# calculate a travel time matrix
ttm <- travel_time_matrix(r5r_core = r5r_core,
                          origins = poi,
                          destinations = poi,
                          mode = mode,
                          departure_datetime = departure_datetime,
                          max_walk_time = max_walk_time,
                          max_trip_duration = max_trip_duration)

head(ttm)

from_id,to_id,travel_time_p50
<chr>,<chr>,<int>
public_market,public_market,0
public_market,bus_central_station,13
public_market,gasometer_museum,10
public_market,santa_casa_hospital,15
public_market,townhall,3
public_market,piratini_palace,17
